# Open Weather Map API

In [1]:
import pandas as pd

Don't forget that for OpenWeatherAPI you need an API key

In [7]:
!pip install pymysql
import pymysql
import sqlalchemy 
import requests

In [20]:
# please use your own API key
OWM_key = 'c4c94a570e01185b8c5aca7295cbafd0'

OpenWeatherMap API has a nice documentation and you can find it here: https://openweathermap.org/api

We are going to use 5 day / 3 hour forecast https://openweathermap.org/forecast5

Provided at their docmentation you can find an example of API call with only having info about city name and country_code
https://openweathermap.org/forecast5#name5

`api.openweathermap.org/data/2.5/forecast?q={city name},{country code}&appid={API key}`

- city name and country code have to be in format e.g. ('Berin,DE')
- appid is your unique API key
- you can also add units which can be 'standard', 'metric', 'imperial'
- or lang for languages e.g. 'de', 'it' etc.
- or cnt for number of timestamps in response

You can also find an example of API call with having info about latitude and longitude  
https://openweathermap.org/forecast5#geo5

api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

You can choose your own approach. We will show here the approach with city name and country code

In [56]:
import pandas as pd

# initialize list of lists
data = [[1, 'Berlin', 'DE'], [2, 'Hamburg', 'DE'], [3, 'London','GB'], [4, 'Manchester','GB'], [5,'Munich','DE'],[6, 'Manchester','GB']]

# Create the pandas DataFrame
cities= pd.DataFrame(data, columns=['city_id', 'city', 'country_code'])

# print dataframe.
cities

,city_id,city,country_code
0,1,Berlin,DE
1,2,Hamburg,DE
2,3,London,GB
3,4,Manchester,GB
4,5,Munich,DE
5,6,Manchester,GB


In [57]:
import requests
url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['city'][0]},{cities['country'][0]}
      &appid={OWM_key}&units=metric"
response = requests.get(url)


In [58]:

url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['city'][0]},{cities['country_code'][0]}
      &appid={OWM_key}&units=metric"
response = requests.get(url)
response.json()


In [59]:
response = requests.get(url)
response.status_code

200

#### 3.2.2 Exploring the response (json)

You can copy paste your json response output to one of many online json viewers to see your json in nicer and more practical way
http://jsonviewer.stack.hu/

In [60]:
response.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1665165600,
   'main': {'temp': 13.81,
    'feels_like': 12.94,
    'temp_min': 13.81,
    'temp_max': 13.91,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 1014,
    'humidity': 65,
    'temp_kf': -0.1},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}],
   'clouds': {'all': 0},
   'wind': {'speed': 2.62, 'deg': 191, 'gust': 5.97},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2022-10-07 18:00:00'},
  {'dt': 1665176400,
   'main': {'temp': 13.46,
    'feels_like': 12.53,
    'temp_min': 12.76,
    'temp_max': 13.46,
    'pressure': 1012,
    'sea_level': 1012,
    'grnd_level': 1013,
    'humidity': 64,
    'temp_kf': 0.7},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'clouds': {'all': 27},
   'wind': {'speed': 2.58, 'deg': 208, 'gust': 6.65},
   'visibility': 1

In [61]:
#This will only work in jupyter notebook so if you are using colab you can skip it

from IPython.display import JSON
# JSON() helps us preview the json in scalable way
JSON(response.json())
# under the node 'list' are the actual weather informations for each hour
JSON(response.json()['list'])

<IPython.core.display.JSON object>

#### 3.2.3 Gathering the data we need from json

In [70]:


weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'feels_like': [],
                'wind_speed': [],
                'pop': []}
# iterating over items in 'list' node and selecting the subnodes gives us the exact info we need 
for hour in response.json()['list']:
  weather_dict['city_id'].append(cities['city_id'][0])
  weather_dict['temperature'].append(hour['main']['temp'])
  weather_dict['wind_speed'].append(hour['wind']['speed'])
  weather_dict['forecast_time'].append(hour['dt_txt'])
  weather_dict['outlook'].append(hour['weather'][0]['description'])
  weather_dict['feels_like'].append(hour['main']['feels_like'])
  weather_dict['pop'].append(hour['pop'])
weather_df = pd.DataFrame(weather_dict)


#### 3.2.4 Creating a new weather DataFrame

In [71]:
weather_df = pd.DataFrame(weather_dict)
weather_df

,city_id,forecast_time,outlook,temperature,feels_like,wind_speed,pop
0,6,2022-10-07 18:00:00,clear sky,13.81,12.94,2.62,0.00
1,6,2022-10-07 21:00:00,scattered clouds,13.46,12.53,2.58,0.00
2,6,2022-10-08 00:00:00,broken clouds,12.62,11.79,2.41,0.00
3,6,2022-10-08 03:00:00,overcast clouds,11.08,10.25,2.45,0.00
4,6,2022-10-08 06:00:00,broken clouds,10.47,9.58,2.95,0.00
5,6,2022-10-08 09:00:00,broken clouds,14.45,13.52,4.99,0.00
6,6,2022-10-08 12:00:00,overcast clouds,14.11,13.35,5.74,0.00
7,6,2022-10-08 15:00:00,broken clouds,15.71,14.62,5.61,0.26
8,6,2022-10-08 18:00:00,scattered clouds,12.38,11.27,4.67,0.00
9,6,2022-10-08 21:00:00,scattered clouds,10.58,9.55,4.37,0.00


In [72]:
schema="gans_database"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="SharpGajanan9860" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [73]:
weather_df.to_sql('weathers',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)

40